# Introducción

In [ ]:
from pyspark.sql.functions import col, avg,when,sum, round

Leemos el archivo CSV mediante PySpark mediante la función read.csv de PySpark. Lo mostramos mediante la función "show()". Veremos que el conjunto de datos que nos muestra está desordenado, lo arreglamos en la siguiente celda.

In [ ]:
df = spark.read.csv('/FileStore/tables/datos.csv',sep = ';')
df.show()

Volvemos a leer los datos pero con "header=True para que sepa que contiene los nombres de las columnas. Implementamos "inferSchema = True" para que PySpark procese el tipo de datos de las columbas de acuerdo a los valores de las filas 

In [ ]:
df = spark.read.csv('/FileStore/tables/datos.csv',sep = ';', header = True, inferSchema =True)
df.show(5)

In [ ]:
#Mostramos las 5 primeras filas
display(df.take(5))

In [ ]:
df.printSchema()

Seleccionamos mediante "select" las columnas "Hospital" y "Facturacion" agrupándolas por "Hospital" y calculamos la media de los valores de "Facturación". 
Después utilizamos la función "groupby" para agrupar los los datos del df por la columna "Hospital"

Con la función "agg" agregamos la columna "Facturacion" con respecto a "Hospital"

TL;DR: Calculamos la media de facturación de cada hospital y ordenamos los resultados segun el nombre del hospital


In [ ]:
display(df.select("Hospital","Facturacion").groupBy("Hospital").agg(avg("Facturacion")).sort("Hospital"))

Mostramos la estructura de los datos para ver su organización

In [ ]:
df.describe().show()

# Missing Values

Podemos ver como hay datos nulos. Veamos que podemos hacer con el comando .na.drop(how=)

In [ ]:
df.show(25)

In [ ]:
# -how = 'any' elimina todas las filas en las que haya un valor null
# -how = 'all' elimina las filas cuyos valores sean todos nulos


df.na.drop(how = 'any').show(25) 

También existe el parámetro thresh = n (int) el cual combinado con how = 'any' mostrará todos las filas que tenga n o más valores distintos de null

In [ ]:
df.na.drop(how = 'any', thresh = 2).show(25) 

Por otro lado podemos rellenar los missing

In [ ]:
df.na.fill({'Name':'Roberto', 'Tiempo_en_hospital_horas':10, 'Edad':36}).show(25) 

Por último, vamos a sobreescribir la tabla de datos para eliminar los null para el resto del trabajo

In [ ]:
df = df.na.drop(how ='any')

# Crear columnas nuevas
Vamos a dividir la columna Tiempo_en_hospital_horas por 24 para calcular en una nueva columna la cantidad de dias que han estado los pacientes en el hospital

In [ ]:
df2 =df.withColumn('Dias_en_el_hospital', round(df['Tiempo_en_hospital_horas']/24, 2)).show()
df2.show()

Creamos la columna Facturacion_por_hora en una nueva variable y redondeamos el resultado

In [ ]:
df3 = df2.withColumn('Facturacion_por_hora', round(df2['Facturacion']/df2['Tiempo_en_hospital_horas'], 2))
df3.show()

Creamos la columna Tiene seguro en una nueva variable y la convertimos en booleana con when(df3.Facturacion_por_hora<140, 0).otherwise(1))


In [ ]:
df4 = df3.withColumn('Tiene_seguro', when(df3.Facturacion_por_hora<140, 0).otherwise(1))
df4.show()

# GroupBy con PySpark
Funciona igual que con pandas. Vamos a ver cuantos pacientes han ido a cada hospital

In [ ]:
df.groupBy('Hospital').count().show()

Mostremos ahora el total de horas y de facturación por hospital

In [ ]:
df.groupBy('Hospital').sum('Tiempo_en_hospital_horas','Facturacion').show()

Mostremos por último la media de edad de los pacientes de cada hospital

In [ ]:
df.groupBy('Hospital').avg('Edad').show()